In [1]:
import pandas as pd
import numpy as np
import os

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

plt.style.use('bmh')


In [2]:

train_d=pd.read_csv('train_agri.csv')

test_d=pd.read_csv('test_agri.csv')

sample=pd.read_csv('submission.csv')

In [3]:
train_d.head(10)
print('Shape of training data is {}'.format(train_d.shape))

print('-------------'*5)

test_d.head()
print('Shape of test data is {}'.format(test_d.shape))

# print('--------------'*5)

# sample.head()
# sample.shape


Shape of training data is (4599, 10)
-----------------------------------------------------------------
Shape of test data is (1199, 9)


In [5]:
train_d['Crop_Damage'].value_counts()

0    3820
1     664
2     115
Name: Crop_Damage, dtype: int64

In [6]:
# Missing value in the dataset

train_d.isnull().sum()
test_d.isnull().sum()

ID                           0
Estimated_Insects_Count      0
Crop_Type                    0
Soil_Type                    0
Pesticide_Use_Category       0
Number_Doses_Week            0
Number_Weeks_Used          119
Number_Weeks_Quit            0
Season                       0
dtype: int64

In [10]:
import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

In [11]:
# value count for the target variable.
train_d.Crop_Damage.value_counts().iplot(kind='bar',yTitle='Count',title='Crop Damaged',color=['red'])


In [12]:
# Crop type which is more prone to pests
train_d.groupby('Crop_Type')['Estimated_Insects_Count'].sum().iplot(kind='bar',xTitle='Type of Crop',yTitle='Count')

In [13]:
# Soil type which require more pestiside

train_d.groupby('Soil_Type')['Estimated_Insects_Count'].sum().iplot(kind='bar',xTitle='Type of Soil',yTitle='Count',color=['blue'])

In [14]:
# Distribution of Estimated_Insects_Count
train_d['Estimated_Insects_Count'].iplot(kind='hist',color=['green'],xTitle='Estimated_Insects_Count')

In [15]:
# Check the presence of outliers in Estimated_Insects_Count

train_d['Estimated_Insects_Count'].iplot(kind='box',color=['pink'],xTitle='Estimated_Insects_Count')

In [16]:
# distribution of number of weeks used
train_d['Number_Weeks_Used'].iplot(kind='hist',xTitle='Number_Weeks_Used',yTitle='count',title='Distribution for Week_used')

In [17]:
# join test and train data
train_d['train_or_test']='train'
test_d['train_or_test']='test'
df=pd.concat([train_d,test_d])
df.head()


,ID,Estimated_Insects_Count,Crop_Type,Soil_Type,Pesticide_Use_Category,Number_Doses_Week,Number_Weeks_Used,Number_Weeks_Quit,Season,Crop_Damage,train_or_test
0,F00000001,188,1,0,1,0,0.0,0,1,0.0,train
1,F00000003,209,1,0,1,0,0.0,0,2,1.0,train
2,F00000004,257,1,0,1,0,0.0,0,2,1.0,train
3,F00000005,257,1,1,1,0,0.0,0,2,1.0,train
4,F00000006,342,1,0,1,0,0.0,0,2,1.0,train


In [18]:
df.isnull().sum()

ID                            0
Estimated_Insects_Count       0
Crop_Type                     0
Soil_Type                     0
Pesticide_Use_Category        0
Number_Doses_Week             0
Number_Weeks_Used           561
Number_Weeks_Quit             0
Season                        0
Crop_Damage                1199
train_or_test                 0
dtype: int64

In [19]:
df['Number_Weeks_Used'].mode()[0]

20.0

In [20]:
df['Number_Weeks_Used']=df['Number_Weeks_Used'].fillna(df['Number_Weeks_Used'].mode()[0])
df.isnull().sum()


ID                            0
Estimated_Insects_Count       0
Crop_Type                     0
Soil_Type                     0
Pesticide_Use_Category        0
Number_Doses_Week             0
Number_Weeks_Used             0
Number_Weeks_Quit             0
Season                        0
Crop_Damage                1199
train_or_test                 0
dtype: int64

In [21]:
df.head()

,ID,Estimated_Insects_Count,Crop_Type,Soil_Type,Pesticide_Use_Category,Number_Doses_Week,Number_Weeks_Used,Number_Weeks_Quit,Season,Crop_Damage,train_or_test
0,F00000001,188,1,0,1,0,0.0,0,1,0.0,train
1,F00000003,209,1,0,1,0,0.0,0,2,1.0,train
2,F00000004,257,1,0,1,0,0.0,0,2,1.0,train
3,F00000005,257,1,1,1,0,0.0,0,2,1.0,train
4,F00000006,342,1,0,1,0,0.0,0,2,1.0,train


In [22]:
df.columns

Index(['ID', 'Estimated_Insects_Count', 'Crop_Type', 'Soil_Type',
       'Pesticide_Use_Category', 'Number_Doses_Week', 'Number_Weeks_Used',
       'Number_Weeks_Quit', 'Season', 'Crop_Damage', 'train_or_test'],
      dtype='object')

In [23]:
# Seperate the train and test data before training.

train=df.loc[df.train_or_test.isin(['train'])]
test=df.loc[df.train_or_test.isin(['test'])]
train.drop(columns={'train_or_test'},axis=1,inplace=True)
test.drop(columns={'train_or_test'},axis=1,inplace=True)


In [24]:
train.drop('ID',axis=1,inplace=True)
train.head()


,Estimated_Insects_Count,Crop_Type,Soil_Type,Pesticide_Use_Category,Number_Doses_Week,Number_Weeks_Used,Number_Weeks_Quit,Season,Crop_Damage
0,188,1,0,1,0,0.0,0,1,0.0
1,209,1,0,1,0,0.0,0,2,1.0
2,257,1,0,1,0,0.0,0,2,1.0
3,257,1,1,1,0,0.0,0,2,1.0
4,342,1,0,1,0,0.0,0,2,1.0


In [26]:
train.shape


(4599, 9)

In [27]:
test.drop(['ID','Crop_Damage'],axis=1,inplace=True)
test.head()
test.shape


KeyError: "['ID' 'Crop_Damage'] not found in axis"

In [28]:
#Convert to X and Y
X_train, Y = train.drop(["Crop_Damage"], axis=1).values, train["Crop_Damage"].values
X_test = test.values

X_train.shape, Y.shape, X_test.shape

((4599, 8), (4599,), (1199, 8))

In [29]:
#Model creation
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from lightgbm import LGBMClassifier
from sklearn.metrics import f1_score
from imblearn.over_sampling import SMOTE
from catboost import CatBoostClassifier

from xgboost import XGBClassifier
from xgboost import plot_importance

from collections import Counter
import warnings
warnings.filterwarnings("ignore")
num_class1, num_class2, num_class3 = Counter(Y)[0], Counter(Y)[1], Counter(Y)[2]

print(num_class1)
print(num_class2)
print(num_class3)

3820
664
115


In [30]:
Counter(Y)

Counter({0.0: 3820, 1.0: 664, 2.0: 115})

In [31]:
#LGBM
kfold, scores = KFold(n_splits=5, shuffle=True, random_state=0), list()
for train, test in kfold.split(X_train):
    x_train, x_test = X_train[train], X_train[test]
    y_train, y_test = Y[train], Y[test]
    
    num_class1, num_class2, num_class3 = Counter(y_train)[0], Counter(y_train)[1], Counter(y_train)[2]
    sm = SMOTE(random_state=27, sampling_strategy={0: int(1.3*num_class1), 1: int(7*num_class2), 2: int(3.2*num_class3)})
    x_train, y_train = sm.fit_resample(x_train, y_train)
    
    model = LGBMClassifier(random_state=27, max_depth=6, n_estimators=400)
    model.fit(x_train, y_train)
    preds = model.predict(x_test)
    score = f1_score(y_test, preds, average="weighted")
    scores.append(score)
    print(score)
print("Average: ", sum(scores)/len(scores))

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
0.7875627793579846
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
0.7643899797413121
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
0.7935457599975068
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
0.7883032873806999
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
0.7789842134607223
Average:  0.7825572039876452


In [32]:
#Make final prediction using LGBM
sm = SMOTE(random_state=27, sampling_strategy={0: int(1.3*num_class1), 1: int(7*num_class2), 2: int(3.2*num_class3)})
x_train, y_train = sm.fit_resample(x_train, y_train)

model1 = LGBMClassifier(random_state=27, max_depth=6, n_estimators=800)
model1.fit(X_train, Y)
preds = model1.predict_proba(X_test)


[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).


In [33]:
preds[0]

array([8.82319658e-02, 9.11767889e-01, 1.45321944e-07])

In [35]:
#Making prediction using Catboost
kfold, scores = KFold(n_splits=5, shuffle=True, random_state=0), list()
for train, test in kfold.split(X_train):
    x_train, x_test = X_train[train], X_train[test]
    y_train, y_test = Y[train], Y[test]
    
    model = CatBoostClassifier(random_state=27, max_depth=4, task_type="CPU", devices="0:1", n_estimators=1000, verbose=500)
    model.fit(x_train, y_train)
    preds_t = model.predict(x_test)
    score = f1_score(y_test, preds_t, average="weighted")
    scores.append(score)
    print(score)
print("Average: ", sum(scores)/len(scores))

Learning rate set to 0.084562
0:	learn: 1.0023570	total: 414ms	remaining: 6m 53s
500:	learn: 0.3744699	total: 3.81s	remaining: 3.8s
999:	learn: 0.3294631	total: 8.65s	remaining: 0us
0.7888325086851247
Learning rate set to 0.084562
0:	learn: 1.0010368	total: 19.1ms	remaining: 19.1s
500:	learn: 0.3608607	total: 4.31s	remaining: 4.3s
999:	learn: 0.3171127	total: 7.63s	remaining: 0us
0.7906830516808878
Learning rate set to 0.084562
0:	learn: 1.0015489	total: 5.9ms	remaining: 5.9s
500:	learn: 0.3595083	total: 4.5s	remaining: 4.48s
999:	learn: 0.3177018	total: 9.26s	remaining: 0us
0.7957945343053009
Learning rate set to 0.084562
0:	learn: 1.0026988	total: 8.38ms	remaining: 8.38s
500:	learn: 0.3725564	total: 3.94s	remaining: 3.93s
999:	learn: 0.3281226	total: 8.9s	remaining: 0us
0.7946610795277622
Learning rate set to 0.084563
0:	learn: 1.0019868	total: 5.69ms	remaining: 5.69s
500:	learn: 0.3598561	total: 4.92s	remaining: 4.89s
999:	learn: 0.3150962	total: 8.58s	remaining: 0us
0.7935549750157

In [38]:
model2 = CatBoostClassifier(random_state=27, task_type="CPU", devices="0:1", n_estimators=1000, max_depth=4, verbose=500)
model2.fit(X_train, Y)
preds1 = model2.predict_proba(X_test)

Learning rate set to 0.08553
0:	learn: 1.0005767	total: 17.4ms	remaining: 17.3s
500:	learn: 0.3754371	total: 5.79s	remaining: 5.76s
999:	learn: 0.3383863	total: 12.1s	remaining: 0us


In [39]:
preds1[0]

array([0.32908684, 0.66431459, 0.00659857])

In [40]:
#Making predictions using XGBoost
#Ensembling both predictions
preds
preds1

array([[3.29086837e-01, 6.64314595e-01, 6.59856871e-03],
       [1.02762312e-01, 8.79585112e-01, 1.76525752e-02],
       [2.25592816e-01, 6.05842920e-01, 1.68564264e-01],
       ...,
       [9.53678649e-01, 4.56279234e-02, 6.93427403e-04],
       [9.55305386e-01, 4.32059138e-02, 1.48870008e-03],
       [9.20768929e-01, 7.88680079e-02, 3.63063316e-04]])

In [41]:
predictions = list()
cb_weight=0.6 # Catboost
lb_weight=0.4 # LGBM
for i, j in zip(preds, preds1):
    xx = [(cb_weight * i[0]) + (lb_weight * j[0]),
          (cb_weight * i[1]) + (lb_weight * j[1]),
          (cb_weight * i[2]) + (lb_weight * j[2])]
    predictions.append(xx)
# print(predictions[:10])
preds_ensemble=np.argmax(predictions,axis=1)

In [42]:
preds_ensemble[:5]

array([1, 1, 0, 0, 0], dtype=int64)

In [45]:
#saving the model

import joblib

  
# Save the model as a pickle in a file 
joblib.dump(preds_ensemble, 'Agriculture.pkl') 


['Agriculture.pkl']